# Neural Network with Principal Components on the Sensor dataset

## Load libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier

## Set directories

In [6]:
directory_hw1 = "/Users/mikepecorino/Documents/machine_learning/HW1/"
directory_hw3 = "/Users/mikepecorino/Documents/machine_learning/HW3/"

## Load inputs

In [9]:
data_all = pd.read_csv(directory_hw3 + "sensor_princomp_nn_input_with_cluster.csv")

## Train/Test split

In [10]:
train = data_all[data_all["tag"].isin(["train", "valid"])]
test = data_all[data_all["tag"] == "test"]

## Define features and response variable

### Features list

In [11]:
features = data_all.columns[data_all.columns.isin(["response", "tag"]) == False]

### Features data

In [13]:
data_all_features = data_all[features]
train_features = train[features]
test_features = test[features]

### Response variable

In [14]:
response = "response"

### Response data

In [15]:
data_all_response = data_all[response]
train_response = train[response]
test_response = test[response]

## Get best neural net hyperparameters from HW1

In [16]:
neural_network_results = pd.read_csv(directory_hw1 + "sensor_neural_net_grid_search.csv")
out_fold_cv_summary = neural_network_results.groupby(["hidden_layer_size", "solver", "activation", "alpha"])["out_fold_accuracy"].agg(["mean"]).sort_values(by = ["mean"], ascending = False)
optimal_results = out_fold_cv_summary.index[np.argmax(out_fold_cv_summary["mean"])]
hidden_layer_size_opt = optimal_results[0]
solver_opt = optimal_results[1]
activation_opt = optimal_results[2]
alpha_opt = optimal_results[3]

## Optimizing the number of components

In [22]:
#Initialize an empty data frame for recording results
neural_network_results = pd.DataFrame(columns = ["hidden_layer_size",
                                                 "solver",
                                                 "activation",
                                                 "alpha",
                                                 "train_accuracy",
                                                 "test_accuracy",
                                                 "time"
                                                 ])

#Get the random projection
start_time = time.time()

#Get the neural net object
neural_network = MLPClassifier(solver = solver_opt, #lbfgs, adam, sgd
                               activation = activation_opt, #identity, logistic, tanh, relu
                               alpha = alpha_opt,
                               hidden_layer_sizes = (hidden_layer_size_opt,),
                               batch_size = "auto",
                               learning_rate = "constant",
                               learning_rate_init = 0.001,
                               power_t = 0.5,
                               max_iter = 200,
                               shuffle = True,
                               random_state = 28,
                               tol = 0.0001,
                               verbose = False,
                               warm_start = False,
                               momentum = 0.9,
                               nesterovs_momentum = True,
                               early_stopping = True,
                               validation_fraction = 0.1,
                               beta_1 = 0.9,
                               beta_2 = 0.999,
                               epsilon = 1e-08,
                               n_iter_no_change = 10,
                               max_fun = 15000)
    
#Fit the model
neural_network.fit(train_features, train_response) 
    
#Evaluate the model
#In fold
train_accuracy = neural_network.score(train_features, train_response)
#Out of fold
test_accuracy = neural_network.score(test_features, test_response)

end_time = time.time()
total_time = end_time - start_time

#Add to results list
neural_network_results = neural_network_results.append({"hidden_layer_size": hidden_layer_size_opt,
                                                        "solver": solver_opt,
                                                        "activation": activation_opt,
                                                        "alpha": alpha_opt,
                                                        "train_accuracy": train_accuracy,
                                                        "test_accuracy": test_accuracy,
                                                        "time": total_time},
                                                        ignore_index = True)
print(neural_network_results, "\n")

  hidden_layer_size solver activation  alpha  train_accuracy  test_accuracy  \
0              1000   adam       relu   0.02        0.645267        0.67187   

       time  
0  1.163226   

